# Python and C++ extension

## Non Linear

In [ ]:
import sys
sys.path.append('/content/CppToPython')

In [ ]:
import numpy as np
import GeDiM4Py as gedim

### Initialize

In [ ]:
lib = gedim.ImportLibrary("/content/CppToPython/release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## Nonlinear Elliptic Problem POD vs PINNs
"Finta equazione di Burger perchè la facciamo in una sola componente"

The aim of our project is to solve the following parametrized problem on the two-dimensional spatial domain $\bar{\Omega} = [0, 1] \times [0, 1]$: given $\mathbf{\mu}=(\mu_0, \mu_1)\in\mathcal{P}=[0.1,1]\times[0.1,1]$, find $u(\mathbf{\mu})$ such that

$$
\begin{cases}
\displaystyle - \Delta u(\mathbf{\mu}) + \frac{\mu_0}{\mu_1} \left(e^{\mu_1\,u(\mathbf{\mu})}-1\right) = g(\mathbf{x};\mathbf{\mu}) & \text{in } \Omega\\
u = 0.0 & \text{in } ∂Ω
\end{cases}
$$

where the source term is defined as $g(\mathbf{x};\mathbf{\mu})=100\,sin(2\pi x_0)\,cos(2\pi x_1)\quad \forall\,\mathbf{x}=(x_0, x_1) \in\Omega$ 

The weak form of the problem becomes, find $u \in V := H^1_0(\Omega)$
$$
\int_{\Omega} \nabla u\cdot\nabla v + \int_{\Omega} \left(e^{\mu_1\,u(\mathbf{\mu})}-1\right) v - \int_{\Omega} g\, v = 0 \quad \forall v \in V \Leftrightarrow f(u; v) := f_1(u; v) + f_2(u; v) + f_3(u; v) = 0 \quad \forall v \in V
$$

Using Newton schema, we solve for each $k$ iteration the problem
$$
J_f [\partial u]_{|_{u_k}} \partial u = - f(u_k; v) = 0 \quad \forall v \in V
$$
where $J_f [\partial u]_{|_{u_k}}$ is the evaluation of the derivative (Jacobian) of $J_f$ in the point $u_k$ along the unknown direction of $\partial u$.

Notice that the functions $f_1(u; v)$ and $f_3(u; v)$ are linear in $u$ and their Gateux derivatives are given respectively by

$$
    J_{f_1} [\partial u]_{|_{u_k}}=f_1(u; v)=\int_{\Omega} \nabla \partial u \cdot \nabla v \qquad\text{and}\qquad J_{f_3} [\partial u]_{|_{u_k}}=0
$$

On the other hand $f_3(u; v)$ is non-linear in u, and its Gateaux derivative is given by

\begin{align*}
    J_{f_2} [\partial u]_{|_{u_k}}&=\lim_{h\to0}\frac{f_2(u_k+\partial u\, h)-f(u_k)}{h}=\\
    \\
    &=\frac{\mu_0}{\mu_1}\lim_{h\to0}\frac{1}{h}\int_{\Omega} \left(e^{\mu_1\,(u_k+\partial u\, h)}-1\right) v-\int_{\Omega} \left(e^{\mu_1\,u_k}-1\right) v=\\
    \\
    &=\frac{\mu_0}{\mu_1}\lim_{h\to0}\frac{1}{h}\int_{\Omega} e^{\mu_1\,u_k}\left(e^{\mu_1\,\partial u\, h}-1\right) v=\\
    \\
    &=\mu_0\int_\Omega e^{\mu_1\,u_k}\, \partial u\,v 
\end{align*}

which is a reaction term.

After computations, we find the linear system, on each $k$ iteration, fixed $u_k$ find $\partial u$ s.t.

$$
\int_{\Omega} \nabla \partial u \cdot \nabla v + \mu_0\int_\Omega e^{\mu_1\,u_k}\, \partial u\,v  = - \int_{\Omega} \underbrace{\nabla u_k}_{g_1} \cdot \nabla v - \int_{\Omega} \underbrace{\left(e^{\mu_1\,u_k}-1\right)}_{g_2} v + \int_{\Omega} g\, v 
$$
In the left hand side we have the sum of the classic diffusion, reaction and advection terms. In the right hand side we have the function $f(u; v)$ evaluated in $u_k$.

## 1. Solve the problem by means of POD-Galerkin method over a Finite Element full order model
# 1.1 **Test**

 In order to implement an error study of the code we will implement, firstly we solve the Nonlinear Elliptic problem by means of POD-Galerkin method over a Finite Element full order model by choosing the exact solution $u_{ex}(\mathbf{x}; \mathbf{\mu}) = 16\,x_0\,x_1\,(1-x_0)(1-x_1)$ and computing the respective right hand side define as

 $$
 g_{ex}(\mathbf{x};\mathbf{\mu}) = -\Delta u_{ex}(\mathbf{x};\mathbf{\mu})+ \frac{\mu_0}{\mu_1} \left(e^{\mu_1\,u_{ex}(\mathbf{x};\mathbf{\mu})}-1\right) = 32 (x_0(1-x_0)+x_1(1-x_1))+ \frac{\mu_0}{\mu_1} \left(e^{\mu_1\,16\,x_0\,x_1\,(1-x_0)(1-x_1)}-1\right)
 $$
 
 Then we will apply our code to the problem introduced above.

In [ ]:
#the numPoints are the quadrature points in which we evaluate the basis function

########## Coefficient of STIFFNESS MATRIX ##########
def Diffusion(numPoints, points):
	values_d = np.ones(numPoints, order='F')
	return values_d.ctypes.data
#####################################################

########## Functions for REACTION MATRIX ##########
# Coefficient
def Reaction(numPoints, points):
	values_r = np.ones(numPoints, order='F')
	return values_r.ctypes.data

# Coefficient depends on u_k
def Reaction_non_linear(numPoints, points, u):
	vecu = gedim.make_nd_array(u, numPoints, np.double) #evaluation of u in quadrature points of our cell
	values_nl = np.exp(mu_1 * vecu)
	return values_nl.ctypes.data
#####################################################

########## Functions for RIGHT HAND SIDE ##########
# Classical right hand side g_ex computed in order to have u(x0,x1)=16*x0*x1*(1-x0)*(1-x1) as the exact solution of the nonlinear elliptic problem
def RHS_g_ex(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values_gex = 32.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) + matPoints[0,:] * (1.0 - matPoints[0,:])) + \
	 +  mu_0 / mu_1 * (np.exp(mu_1 * matPoints[0,:] * matPoints[1,:] * (1.0 - matPoints[0,:]) * (1.0 - matPoints[1,:]))-1)
	return values_gex.ctypes.data

# Right hand side g given from the problem
def RHS_g(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values_g = 100.0 * np.sin(2 * np.pi * matPoints[0,:]) * np.cos(2 * np.pi * matPoints[1,:])
	return values_g.ctypes.data

# Non linear term evaluated in u_k
def RHS_non_linear_g2(numPoints, points, u):
	vecu = gedim.make_nd_array(u, numPoints, np.double) #evaluation of u in quadrature points of our cell
	values_nl = np.exp(mu_1 * vecu)-1
	return values_nl.ctypes.data

# Diffusion term evaluated in u_k
def RHS_non_linear_g1(numPoints, points, u, u_x, u_y): #matrix that contain the gradient of our solution
	vecu_x = gedim.make_nd_array(u_x, numPoints, np.double)
	vecu_y = gedim.make_nd_array(u_y, numPoints, np.double)
	values_nl_d_f = np.zeros((2, numPoints), order='F') # we have always the geometric dimension in the rows, and number of points in the columns
	values_nl_d_f[0,:] = vecu_x
	values_nl_d_f[1,:] = vecu_y
	return values_nl_d_f.ctypes.data
#####################################################

################## EXACT SOLUTION ###################
# Evaluation in quadrature points
def Test_exactSolution(numPoints, points): 
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values_ex = 16.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:]))
	return values_ex.ctypes.data

# Derivatives in each direction
def Test_exactDerivativeSolution(direction, numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)

	if direction == 0:
		values_ex_d = 16.0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:])
	elif direction == 1:
		values_ex_d = 16.0 * (1.0 - 2.0 * matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:])
	else:
		values_ex_d = np.zeros(numPoints, order='F')

	return values_ex_d.ctypes.data
#####################################################


def Ones(numPoints, points):
	values_one = np.ones(numPoints, order='F')
	return values_one.ctypes.data

def OnesDerivative(numPoints, points):
	values_one_d = np.ones((2, numPoints), order='F')
	return values_one_d.ctypes.data

def Zeros(numPoints, points): 
	values_zero = np.zeros(numPoints, order='F')
	return values_zero.ctypes.data

def ZerosDerivative(direction, numPoints, points):  #gradient null in each direction
	values_zero_d = np.zeros(numPoints, order='F')
	return values_zero_d.ctypes.data

### Define Simulation Parameters

Set geometry parameters

In [ ]:
meshSize = 0.01 #We choose a meshsize = maximum area required in our domain discretization
order = 1 #Order of finite element

#all the boundary condition are homogenous dirichlte boundary condition so we use just one marker for the border equal to 1
domain = { 'SquareEdge': 1.0, 'VerticesBoundaryCondition': [1,1,1,1], 'EdgesBoundaryCondition': [1,1,1,1], 'DiscretizationType': 1, 'MeshCellsMaximumArea': meshSize } 
[meshInfo, mesh] = gedim.CreateDomainSquare(domain, lib)

discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 2] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

### **Newton Algorithm**

Set Newton parameters

In [ ]:
residual_norm = 1.0; # variable in which we sotre the residual L2 norm
solution_norm = 1.0; # variable in which we store the solution L2 norm. They are needed in order to implement a Newton step
newton_tol = 1.0e-6 # tolerance we want to achieve with the solution approximation. It cannot be lower than the machine precision (epsilon di macchina)
max_iterations = 20 # 7 # if the mesh is not too fine it is not necessary a too high number of iteration
num_iteration = 1 

Set Initial Solution

In [ ]:
# Newton schema is a second order algorithm, ...(min.14)
u_k = np.zeros(problemData['NumberDOFs'], order='F') 
u_strong = np.zeros(problemData['NumberStrongs'], order='F')

## **POD**

Let us define the training set for the POD

In [ ]:
### define the training set

snapshot_num = 300
mu1_range = [0.1, 1.]
mu2_range = [0.1, 1.]
P = np.array([mu1_range, mu2_range])

training_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(snapshot_num, P.shape[0]))

We can now proceed with the snapshot matrix creation. However, we need to be careful: the problem is not affine in the parameters and we need to assemble the Reaction matrix and the nonlinear forcing term for each parametric instance.

In [ ]:
################################ SNAPSHOT MATRIX ##############################
snapshot_matrix = []

tol = 1. - 1e-7
N_max = 10

for mu in training_set:
    mu_0 = mu[0]
    mu_1 = mu[1]

    while num_iteration < max_iterations and residual_norm > newton_tol * solution_norm: 
        
        # Stiffness matrix
        [stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Diffusion, problemData, lib)

        inner_product = stiffness

        # Reaction matrix
        [reaction1, reactionStrong1] = gedim.AssembleNonLinearReactionMatrix(Reaction, Reaction_non_linear, u_k, u_strong, problemData, lib)

        reaction = mu_0 * reaction

        # Forcing term associate to the chosen exact solution
        forcingTerm_g_ex = gedim.AssembleForcingTerm(RHS_g_ex, problemData, lib)

        # Diffusion term evaluated in u_k
        forcingTerm_der_v = gedim.AssembleNonLinearDerivativeForcingTerm(OnesDerivative, RHS_non_linear_g1, u_k, u_strong, problemData, lib)
    
        # Nonlinear term evaluated in u_k
        forcingTerm_v = gedim.AssembleNonLinearForcingTerm(Ones, RHS_non_linear_g2, u_k, u_strong, problemData, lib)

        
        du = gedim.LUSolver(stiffness + reaction, \
                forcingTerm_g_ex - forcingTerm_v - forcingTerm_der_v, \
                lib)
    
        # Newton step: we add to the old solution the increment found with the Newton method
        u_k = u_k + du

        #Error computation: con il galerkin classico possiamo calcolare l'errore con le funzioni ComputeError, ma se il metodo non è quello classico, non è detto esistano le rispettive funzioni
        du_normL2 = gedim.ComputeErrorL2(Zeros, du, np.zeros(problemData['NumberStrongs'], order='F'), lib)
        u_errorL2 = gedim.ComputeErrorL2(Test_exactSolution, u_k, u_strong, lib)
        u_errorH1 = gedim.ComputeErrorH1(Test_exactDerivativeSolution, u_k, u_strong, lib) 
        u_normL2 = gedim.ComputeErrorL2(Zeros, u_k, u_strong, lib) # we aim it is zero, so we pass to it the zero function
        u_normH1 = gedim.ComputeErrorH1(ZerosDerivative, u_k, u_strong, lib) #Similarly t the previous, we pass the function with derivative null along each direction
    
        solution_norm = u_normL2;
        residual_norm = du_normL2;
    
        print("dofs", "h", "errorL2", "errorH1", "residual", "iteration", "max_iteration")
        print(problemData['NumberDOFs'], '{:.16e}'.format(problemData['H']), '{:.16e}'.format(u_errorL2 / u_normL2), '{:.16e}'.format(u_errorH1 / u_normH1), '{:.16e}'.format(residual_norm / u_normL2), '{:d}'.format(num_iteration), '{:d}'.format(max_iterations)) 
    
        num_iteration = num_iteration + 1
    
    snapshot_matrix.append(np.copy(u_k))

snapshot_matrix = np.array(snapshot_matrix)


Let us build and analyze the covariance matrix.

In [ ]:
C = snapshot_matrix @ inner_product @ np.transpose(snapshot_matrix) ## metti inner product

# VM, L, VMt = np.linalg.svd((C))

L_e, VM_e = np.linalg.eig(C)
eigenvalues = []
eigenvectors = []


#### check

for i in range(len(L_e)):
  eig_real = L_e[i].real
  eig_complex = L_e[i].imag
  assert np.isclose(eig_complex, 0.)
  eigenvalues.append(eig_real)
  eigenvectors.append(VM_e[i].real)


total_energy = sum(eigenvalues)
retained_energy_vector = np.cumsum(eigenvalues)
relative_retained_energy = retained_energy_vector/total_energy


if all(flag==False for flag in relative_retained_energy>= tol):
  N = N_max
else:
  N = np.argmax(relative_retained_energy >= tol) + 1

print(N)
print(relative_retained_energy)

And now let us build the basis functions and $\mathbb B$.

In [ ]:
# Create the basis function matrix
basis_functions = []
for n in range(N):
  eigenvector =  eigenvectors[n]
  basis = np.transpose(snapshot_matrix) @ eigenvector
  norm = np.sqrt(np.transpose(basis) @ inner_product @ basis)
  basis /= norm
  basis_functions.append(np.copy(basis))

basis_functions = np.transpose(np.array(basis_functions))

### Standard ROMs
If we want to perform standard ROMs we still need to implement the Newton algorithm and assemble the system.

### New parameters

In [ ]:
mu_0 = 0.8
mu_1 = 0.4

In [ ]:
while num_iteration < max_iterations and residual_norm > newton_tol * solution_norm: 
        
    # Stiffness matrix
    [stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Diffusion, problemData, lib)

    # Reaction matrix
    [reaction1, reactionStrong1] = gedim.AssembleNonLinearReactionMatrix(Reaction, Reaction_non_linear, u_k, u_strong, problemData, lib)

    reaction = mu_0 * reaction

    # Forcing term given by the problem
    forcingTerm_g_ex = gedim.AssembleForcingTerm(RHS_g_ex, problemData, lib)

    # Diffusion term evaluated in u_k
    forcingTerm_der_v = gedim.AssembleNonLinearDerivativeForcingTerm(OnesDerivative, RHS_non_linear_g1, u_k, u_strong, problemData, lib)
    
    # Nonlinear term evaluated in u_k
    forcingTerm_v = gedim.AssembleNonLinearForcingTerm(Ones, RHS_non_linear_g2, u_k, u_strong, problemData, lib)

    ##################### REDUCED #########################
    reduced_stiff = np.transpose(basis_functions) @ stiffness @ basis_functions
    reduced_reac = np.transpose(basis_functions) @ reaction @ basis_functions

    reduced_fT_gex =  np.transpose(basis_functions) @ forcingTerm_g_ex
    reduced_fT_der_v =  np.transpose(basis_functions) @ forcingTerm_der_v
    reduced_fT_v =  np.transpose(basis_functions) @ forcingTerm_v

    
    reduced_du = gedim.LUSolver(reduced_stiff + reduced_reac, \
            reduced_fT_gex - reduced_fT_der_v - reduced_fT_v, \
            lib)
    
    # Newton step: we add to the old solution the increment found with the Newton method
    reduced_u_k = reduced_u_k + reduced_du
    
    #Projection of reduced soution u_k
    u_k = basis_functions @ reduced_u_k 

    #Error computation: con il galerkin classico possiamo calcolare l'errore con le funzioni ComputeError, ma se il metodo non è quello classico, non è detto esistano le rispettive funzioni
    du_normL2 = gedim.ComputeErrorL2(Zeros, reduced_du, np.zeros(problemData['NumberStrongs'], order='F'), lib)
    u_errorL2 = gedim.ComputeErrorL2(Test_exactSolution, u_k, u_strong, lib)
    u_errorH1 = gedim.ComputeErrorH1(Test_exactDerivativeSolution, u_k, u_strong, lib) 
    u_normL2 = gedim.ComputeErrorL2(Zeros, u_k, u_strong, lib) # we aim it is zero, so we pass to it the zero function
    u_normH1 = gedim.ComputeErrorH1(ZerosDerivative, u_k, u_strong, lib) #Similarly t the previous, we pass the function with derivative null along each direction
    
    solution_norm = u_normL2;
    residual_norm = du_normL2;
    
    print("dofs", "h", "errorL2", "errorH1", "residual", "iteration", "max_iteration")
    print(problemData['NumberDOFs'], '{:.16e}'.format(problemData['H']), '{:.16e}'.format(u_errorL2 / u_normL2), '{:.16e}'.format(u_errorH1 / u_normH1), '{:.16e}'.format(residual_norm / u_normL2), '{:d}'.format(num_iteration), '{:d}'.format(max_iterations)) 
    
    num_iteration = num_iteration + 1

### Plot Solution

In [ ]:
#Visualizzazione della soluzione e calcolo dell'errore FONDAMENTALI NEL PROGETTO !!!
gedim.PlotSolution(mesh, dofs, strongs, u_k, u_strong)

#PLotSolution implements in Pyton the two plot of 2D and 3D plots of the solution. IT's better to export the solution in order to visulize the solution in other applications, as ParaView
#ExportSolution give us a file .inp which is the file format of the UCD structur file. It's limitated to specific type of geometrical elemntsù: tringle hexahedron, thetraedron, pyramidon, line, points.
# Filter "Warp by scalar" in ParaView let choos one of the scalar associated to the solution... to 3D visualization
gedim.ExportSolution(Test_exactSolution, u_k, u_strong, lib)

#Nella costruzione delle funzioni non ci interessava sapere chi fossero i nodi di quadratura per la costruzione delle matrici. In questo caso invece, dovendo valutare l'errore in tutte
#le celle dobbiamo conoscere i nodi di quadratura in tutto il dominio e li salviamo nella matrice globale quadraturePoints
#Inoltre per ogni nodo di quadratura dobbiamo conoscere il peso per cui dobbiamo moltiplicare, altrimenti l'approssimazione dell'integrale non è corretta. 
#La sommma dei pesi è 1 poichè approssimo l'integrale della funzione unitaria che fornisce la misura del nostro dominio
[numQuadraturePoints, quadraturePoints, quadratureWeights, sol, sol_x, sol_y] = gedim.EvaluateSolutionOnPoints(u_k, u_strong, lib)

#If the order of FE is 2 the PlotSoution does not use the info on the middle points of the FE but just that on the vertices
#In order to taking account also of the dofs that are not equal to the vertices we use ExtractSolutionOnPoints.
#We have a collection of points and in each point we have a value of the solution: we have the galerkin numerical solution evaluated in each qadrature formula. 
#We don't have the solution on the cells.
#In order to have the solution on cells we have to add a filter to the numeric solution that is associated to each point of the mesh, not to the cell. We need to change cell
#array to point array: using cellDataPointData.
#Then, we have to generate a mesh: we use the Delauneay 2D from the filter on the solution on points. We obtain a mesh conformed to our points.
#The borders are missing because the quadrature formula is implemented INSIDE the cells. We add the warpByScalar filter and we see the expected better approximation of the exact solution
gedim.ExportSolutionOnPoints(numQuadraturePoints, quadraturePoints, sol, lib) 


#If we have a network, notice that sol is an array with shape 864, and for each point we have the evaluation of the solution on each point. In Galerkin it is correct, but not in NN
#In the latter case we have to evaluate the network in quadrature points adding the line 
#nn(QuadraturePoints)
#before the export.

Let us comment a bit on the error analysis and the _speed up_.

In [ ]:
### compute error
import time

abs_err = []
rel_err = []
testing_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(100, P.shape[0]))
speed_up = []

print("Computing error and speedup analysis")

for mu in testing_set:
  
  mu_0 = mu[0]
  mu_1 = mu[1]
  
  #### the problem is not affine: I have to assemble in this stage!! ### Above we don't assemble everything together
  start_assembling = time.time() # we introduce a time assembling
  Dirichlet_top = gedim.AssembleStrongSolution(Dirichlet_Term, 3, problemData, lib) ## label 
  f1_D = stiffnessStrong1 @ Dirichlet_top
  f2_D = stiffnessStrong2 @ Dirichlet_top
  r_f1_D = np.transpose(basis_functions) @ (stiffnessStrong1 @ Dirichlet_top)
  r_f2_D = np.transpose(basis_functions) @ (stiffnessStrong2 @ Dirichlet_top)
  time_assembling =  time.time() - start_assembling

  ##### full #####
  stiffness = thetaA1*stiffness1 + thetaA2*stiffness2
  weakTerm_down = thetaf1*weakTerm_down1
  f = weakTerm_down - (thetaA1*stiffnessStrong1 + thetaA2*stiffnessStrong2) @ Dirichlet_top
  
  
  start_fom = time.time()
  full_solution = gedim.LUSolver(stiffness, f, lib)
  time_fom = time.time() - start_fom

  #### reduced #####

  reduced_rhs = thetaA1*reduced_stiff1 + thetaA2*reduced_stiff2
  reduced_lhs = thetaf1*reduced_w - (thetaA1*r_f1_D + thetaA2*r_f2_D)
  
  start_rom = time.time()
  reduced_solution = np.linalg.solve(reduced_rhs, reduced_lhs)
  time_rom = time.time() - start_rom
  
  speed_up.append(time_fom/(time_rom + time_assembling))
  
  proj_reduced_solution = basis_functions@reduced_solution

  ### computing error

  error_function = full_solution - proj_reduced_solution
  error_norm_squared_component = np.transpose(error_function) @ inner_product @ error_function
  absolute_error = np.sqrt(abs(error_norm_squared_component))
  abs_err.append(absolute_error)
  
  full_solution_norm_squared_component = np.transpose(full_solution) @  inner_product @ full_solution
  relative_error = absolute_error/np.sqrt(abs(full_solution_norm_squared_component))
  rel_err.append(relative_error)
  

# 1.2 **The problem**
From the error study we have seen that our code give a good approximation of the exact solution. We now implement the code in order to find an approximation of the unknown solution of the nonlinear elliptic problem with the forcing term equal to 
$$
g(\mathbf{x};\mathbf{\mu})=100\,sin(2\pi x_0)\,cos(2\pi x_1)\quad \forall\,\mathbf{x}=(x_0, x_1) \in\Omega
$$

In [ ]:
# Right hand side g given from the problem
def RHS_g(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values_g = 100.0 * np.sin(2 * np.pi * matPoints[0,:]) * np.cos(2 * np.pi * matPoints[1,:])
	return values_g.ctypes.data

Set Newton parameters

In [ ]:
residual_norm = 1.0; # variable in which we sotre the residual L2 norm
solution_norm = 1.0; # variable in which we store the solution L2 norm. They are needed in order to implement a Newton step
newton_tol = 1.0e-6 # tolerance we want to achieve with the solution approximation. It cannot be lower than the machine precision (epsilon di macchina)
max_iterations = 20 # 7 # if the mesh is not too fine it is not necessary a too high number of iteration
num_iteration = 1 

Set Initial Solution

In [ ]:
# Newton schema is a second order algorithm, ...(min.14)
u_k = np.zeros(problemData['NumberDOFs'], order='F') 
u_strong = np.zeros(problemData['NumberStrongs'], order='F')

### Newton algorithm

In [ ]:

while num_iteration < max_iterations and residual_norm > newton_tol * solution_norm: 
        
    # Stiffness matrix
    [stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Diffusion, problemData, lib)


    # Reaction matrix
    [reaction1, reactionStrong1] = gedim.AssembleNonLinearReactionMatrix(Reaction, Reaction_non_linear, u_k, u_strong, problemData, lib)

    reaction = mu_0 * reaction

    # Forcing term given by the problem
    forcingTerm_g = gedim.AssembleForcingTerm(RHS_g, problemData, lib)

    # Diffusion term evaluated in u_k
    forcingTerm_der_v = gedim.AssembleNonLinearDerivativeForcingTerm(OnesDerivative, RHS_non_linear_g1, u_k, u_strong, problemData, lib)
    
    # Nonlinear term evaluated in u_k
    forcingTerm_v = gedim.AssembleNonLinearForcingTerm(Ones, RHS_non_linear_g2, u_k, u_strong, problemData, lib)

        
    du = gedim.LUSolver(stiffness + reaction, \
            forcingTerm_g - forcingTerm_v - forcingTerm_der_v, \
            lib)
    
    # Newton step: we add to the old solution the increment found with the Newton method
    u_k = u_k + du
    
    du_normL2 = gedim.ComputeErrorL2(Zeros, du, np.zeros(problemData['NumberStrongs'], order='F'), lib)
    u_normL2 = gedim.ComputeErrorL2(Zeros, u_k, u_strong, lib) 
    u_normH1 = gedim.ComputeErrorH1(ZerosDerivative, u_k, u_strong, lib)

    solution_norm = u_normL2;
    residual_norm = du_normL2;
    
    print("dofs", "h", "residual", "iteration", "max_iteration")
    print(problemData['NumberDOFs'], '{:.16e}'.format(problemData['H']), '{:.16e}'.format(residual_norm / u_normL2), '{:d}'.format(num_iteration), '{:d}'.format(max_iterations)) 
    
    num_iteration = num_iteration + 1


### Plot Solution

In [ ]:
#Visualizzazione della soluzione e calcolo dell'errore FONDAMENTALI NEL PROGETTO !!!
gedim.PlotSolution(mesh, dofs, strongs, u_k, u_strong)

#PLotSolution implements in Pyton the two plot of 2D and 3D plots of the solution. IT's better to export the solution in order to visulize the solution in other applications, as ParaView
#ExportSolution give us a file .inp which is the file format of the UCD structur file. It's limitated to specific type of geometrical elemntsù: tringle hexahedron, thetraedron, pyramidon, line, points.
# Filter "Warp by scalar" in ParaView let choos one of the scalar associated to the solution... to 3D visualization
gedim.ExportSolution(Test_exactSolution, u_k, u_strong, lib)

#Nella costruzione delle funzioni non ci interessava sapere chi fossero i nodi di quadratura per la costruzione delle matrici. In questo caso invece, dovendo valutare l'errore in tutte
#le celle dobbiamo conoscere i nodi di quadratura in tutto il dominio e li salviamo nella matrice globale quadraturePoints
#Inoltre per ogni nodo di quadratura dobbiamo conoscere il peso per cui dobbiamo moltiplicare, altrimenti l'approssimazione dell'integrale non è corretta. 
#La sommma dei pesi è 1 poichè approssimo l'integrale della funzione unitaria che fornisce la misura del nostro dominio
[numQuadraturePoints, quadraturePoints, quadratureWeights, sol, sol_x, sol_y] = gedim.EvaluateSolutionOnPoints(u_k, u_strong, lib)

#If the order of FE is 2 the PlotSoution does not use the info on the middle points of the FE but just that on the vertices
#In order to taking account also of the dofs that are not equal to the vertices we use ExtractSolutionOnPoints.
#We have a collection of points and in each point we have a value of the solution: we have the galerkin numerical solution evaluated in each qadrature formula. 
#We don't have the solution on the cells.
#In order to have the solution on cells we have to add a filter to the numeric solution that is associated to each point of the mesh, not to the cell. We need to change cell
#array to point array: using cellDataPointData.
#Then, we have to generate a mesh: we use the Delauneay 2D from the filter on the solution on points. We obtain a mesh conformed to our points.
#The borders are missing because the quadrature formula is implemented INSIDE the cells. We add the warpByScalar filter and we see the expected better approximation of the exact solution
gedim.ExportSolutionOnPoints(numQuadraturePoints, quadraturePoints, sol, lib) 


#If we have a network, notice that sol is an array with shape 864, and for each point we have the evaluation of the solution on each point. In Galerkin it is correct, but not in NN
#In the latter case we have to evaluate the network in quadrature points adding the line 
#nn(QuadraturePoints)
#before the export.